In [3]:
import pandas as pd
import numpy as np
import sys, os, json
sys.path.insert(1, '../utils')
from GPTJFineTuner import GPTJFineTuner
from regressionDataGen import dataGenerate

In [4]:
def save_log(
    data_dir = '.',
    sim_idx = 0,
    func = 'cosine',
    n_train = 200,
    n_valid = 50,
    p = 1,
    integer = False,
    lb = -10,
    ub = 10,
    noise = 0.1,
    batch_size = 4,
    learning_rate = 1e-4,
    epochs = 50,
    cuda_idx = 0,
    debug_every = False,
    log_path = None,
):
    config = {'learning_rate': learning_rate, 'batch_size': batch_size, 'epochs':epochs,  'weight_decay': 0.01, 'warmup_steps': 6}
    dg = dataGenerate('data')

    with open('%s/data_%d/%s_n_%s_p_%d_int_%d_(%.1f,%.1f)_noise_%.2f_all.json' % (data_dir, sim_idx+1, func, n_train+n_valid, p, integer, lb, ub, noise), 'r') as f:
        data_json = json.load(f)

    train_df = pd.DataFrame(data_json['train_x'])
    train_df['y'] = data_json['train_y']

    valid_df = pd.DataFrame(data_json['validation_x'])
    valid_df['y'] = data_json['validation_y']

    test_df = pd.DataFrame(data_json['test_x'])
    test_df['y'] = data_json['test_y']

    train_prompts = dg.array2prompts(data_json['train_x'])
    valid_prompts = dg.array2prompts(data_json['validation_x'])
    test_prompts = dg.array2prompts(data_json['test_x'])
    
    train_file = '%s/data_%d/%s_n_%s_p_%d_int_%d_(%.1f,%.1f)_noise_%.2f_train.jsonl' % (data_dir,sim_idx+1, func, n_train+n_valid, p, integer, lb, ub, noise)
    valid_file = '%s/data_%d/%s_n_%s_p_%d_int_%d_(%.1f,%.1f)_noise_%.2f_valid.jsonl' % (data_dir,sim_idx+1, func, n_train+n_valid, p, integer, lb, ub, noise)
    #valid_prompts, grid_prompts,X_grid,y_grid,train_file,valid_file 
    gptj_fine_tuner = GPTJFineTuner(config=config,train_jsonl=train_file,valid_jsonl=valid_file,cuda_idx=cuda_idx)
    gptj_fine_tuner.fine_tune()
    loss = gptj_fine_tuner.log_train(
        epochs,
        train_prompts,
        valid_prompts,
        valid_df,
        test_prompts,
        test_df,
        y_name='y', 
        train_df = train_df, 
        log = False,
        debug_every = debug_every
    )
    
    if log_path is None: log_path = '%s/data_%d/%s_n_%s_p_%d_int_%d_(%.1f,%.1f)_noise_%.2f_log.jsonl' % (data_dir,sim_idx+1, func, n_train+n_valid, p, integer, lb, ub, noise)
    with open(log_path, 'w') as f:
        json.dump(loss, f)
    return loss

In [5]:
save_log(func = 'cosine', cuda_idx = 1, debug_every = 5, log_path = 'data_1/log_try.jsonl')

==== Epoch 1.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.63it/s, val_loss=1.66]


==== Epoch 2.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.57it/s, val_loss=1.67]


==== Epoch 3.0000 ====


...

==== Epoch 49.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.56it/s, val_loss=4.04]


==== Epoch 50.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.56it/s, val_loss=3.81]


defaultdict(list,
            {'train_y': [8.104773640292196,
              4.321537311396268,
              8.352766012131024,
              -8.36006253024718,
              4.25720600773544,
              -9.921108545248021,
              -7.878011425532251,
              7.177570287757093,
              -5.546800045104547,
              -7.095592447190019,
              4.931268025120364,
              1.2041329730605086,
              5.871575080074892,
              -4.191827737081201,
              7.3578872553490315,
              -7.567575630375721,
              -7.863464812540323,
              -9.37045304045537,
              -2.166327171070161,
              -6.00732619483904,
              -10.505779741132654,
              -7.756619609091484,
              9.630555441883422,
              8.64512965666377,
              7.2101310443925986,
              -0.04932123295240376,
              0.3431854333351749,
              -5.761938432419466,
              -0.0991723701665

In [5]:
save_log(func = 'cosine', cuda_idx = 1, debug_every = 5, log_path = 'data_1/log_try_lr_1e-5.jsonl', learning_rate = 1e-5)

==== Epoch 1.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.61it/s, val_loss=3.64]


==== Epoch 2.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.56it/s, val_loss=1.88]


==== Epoch 3.0000 ====


...

==== Epoch 49.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.56it/s, val_loss=2.79]


==== Epoch 50.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.53it/s, val_loss=2.89]


defaultdict(list,
            {'train_y': [8.104773640292196,
              4.321537311396268,
              8.352766012131024,
              -8.36006253024718,
              4.25720600773544,
              -9.921108545248021,
              -7.878011425532251,
              7.177570287757093,
              -5.546800045104547,
              -7.095592447190019,
              4.931268025120364,
              1.2041329730605086,
              5.871575080074892,
              -4.191827737081201,
              7.3578872553490315,
              -7.567575630375721,
              -7.863464812540323,
              -9.37045304045537,
              -2.166327171070161,
              -6.00732619483904,
              -10.505779741132654,
              -7.756619609091484,
              9.630555441883422,
              8.64512965666377,
              7.2101310443925986,
              -0.04932123295240376,
              0.3431854333351749,
              -5.761938432419466,
              -0.0991723701665

In [ ]:
save_log(func = 'linear', cuda_idx = 1)
save_log(func = 'quadratic', cuda_idx = 1)
save_log(func = 'l1norm', cuda_idx = 1)
save_log(func = 'piecewise', cuda_idx = 1)
save_log(func = 'cosine', cuda_idx = 1)
save_log(func = 'exponential', cuda_idx = 1)

==== Epoch 1.0000 ====


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  6.62it/s, val_loss=1.35]
